In [2]:
import pandas as pd
import numpy as np
import sklearn
import re
import nltk
import json
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [2]:
dataset_df_train = pd.read_csv("Amazon_book_review_train.csv")
dataset_df_test = pd.read_csv("Amazon_book_review_test.csv")

In [3]:
dataset_df_train.head()

,rating,review_text,review_cleaned
0,5.0,"This book is everything that is simple, delica...",this book is everything that is simple delicat...
1,5.0,When I first started writing poetry at age 12 ...,when i first started writing poetry at age or ...
2,5.0,"Khalil Gibran's book, The Prophet, has the pow...",khalil gibran s book the prophet ha the power ...
3,5.0,I was given this book by a writer friend who c...,i wa given this book by a writer friend who ca...
4,5.0,A book to be treasured. A tremendous poet deal...,a book to be treasured a tremendous poet deali...


In [29]:
dataset_df_train_dropna = dataset_df_train.dropna(axis = 0)
dataset_df_test_dropna = dataset_df_test.dropna(axis = 0)

In [30]:
dataset_df_train.isnull().any()

rating            False
review_text        True
review_cleaned     True
dtype: bool

In [33]:
dataset_df_test.isnull().any()

rating            False
review_text        True
review_cleaned     True
dtype: bool

In [31]:
dataset_df_train_dropna.isnull().any()

rating            False
review_text       False
review_cleaned    False
dtype: bool

In [34]:
dataset_df_test_dropna.isnull().any()

rating            False
review_text       False
review_cleaned    False
dtype: bool

In [39]:
check_cleaned = dataset_df_train['review_cleaned'].isnull()
check_raw = dataset_df_train['review_text'].isnull()
print(dataset_df_train['review_cleaned'][check_cleaned])
print(dataset_df_train['review_text'][check_raw])

12456    NaN
18697    NaN
Name: review_cleaned, dtype: object
12456    NaN
18697    NaN
Name: review_text, dtype: object


In [40]:
dataset_df_train.iloc[dataset_df_train['review_cleaned'][check_cleaned].index]

,rating,review_text,review_cleaned
12456,3.0,NaN,NaN
18697,2.0,NaN,NaN


In [48]:
dataset_df_train['review_cleaned'][dataset_df_train['review_cleaned'] == '']

Series([], Name: review_cleaned, dtype: object)

In [49]:
(dataset_df_train['review_cleaned'] == '').any()

False

In [51]:
#Sometimes the missing string is ''. This kind of missing isnull() cannot detect.
dataset_df_train[dataset_df_train['review_cleaned'] == ''].index

Int64Index([], dtype='int64')

In [42]:
## Put all the review of train_set in a list to generate the features
clean_train_reviews = []
for i in dataset_df_train_dropna["review_cleaned"]:
    clean_train_reviews.append(i)             

In [43]:
## Put all the review of test_set in a list 
clean_test_reviews = []
for i in dataset_df_test_dropna["review_cleaned"]:
    clean_test_reviews.append(i) 

In [44]:
## Generate Features (bag of words)
vectorizer = CountVectorizer(analyzer = "word", ngram_range=(1, 2), tokenizer = None, preprocessor = None, stop_words = None,max_features=2000)
train_data_features = vectorizer.fit_transform(clean_train_reviews)
test_data_features = vectorizer.fit_transform(clean_test_reviews)

In [47]:
forest = RandomForestClassifier(n_estimators = 200) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, dataset_df_train_dropna["rating"])

In [53]:
result = forest.predict(test_data_features)

In [56]:
print('The accurate of Random Forest is {}.'.format((dataset_df_test_dropna["rating"] == result).sum()/len(result)))

The accurate of Random Forest is 0.26030412164865946.


In [16]:
test = pd.DataFrame([{'a': 1, 'b':None, 'c': 'Hi', 'd':'', 'f':np.Inf/np.Inf},{'b':'Ni', 'd':np.NaN, 'f':3}])
print(test)
print(test['f'].dtype)
print(type(test.loc[1,'c']))


     a     b    c    d    f
0  1.0  None   Hi       NaN
1  NaN    Ni  NaN  NaN  3.0
float64
<class 'float'>


In [4]:
test.isnull()

,a,b,c,d,f
0,False,True,False,False,True
1,True,False,True,True,False


In [72]:
test[test['d'] == '']

,a,b,c,d,f
0,1.0,None,Hi,,NaN


In [5]:
test[test['b'] == None]

,a,b,c,d,f


In [10]:
test_fillna = test.fillna(0)
print(test_fillna)

     a   b   c  d    f
0  1.0   0  Hi     0.0
1  0.0  Ni   0  0  3.0


In [11]:
print(test_fillna.iloc[0,1])
type(test_fillna.iloc[0,1])

0


int

In [12]:
print(test_fillna.iloc[1,2])
type(test_fillna.iloc[1,2])

0


int

In [14]:
test_fillna.values

array([[1.0, 0, 'Hi', '', 0.0],
       [0.0, 'Ni', 0, 0, 3.0]], dtype=object)

In [19]:
print(type(test_fillna.values[0,0]))
print(type(test_fillna.values[0,1]))
print(type(test_fillna.values[0,2]))

<class 'float'>
<class 'int'>
<class 'str'>
